<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [50]:
import pandas as pd
import networkx as nx

In [51]:
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

In [52]:
network = pd.read_csv('data/cell_lines_corr-above0-5-0-323.net', sep='\t')
node_lookup = pd.read_csv('data/cell_lines_corr-above0-5-0-323-subtypes.nodes', sep='\t')

In [53]:
list1 = list(px.colors.qualitative.Dark24)
list2 = list(px.colors.qualitative.Light24)
list1.extend(list2)

In [54]:
primary_colormap = dict(zip(list(set(node_lookup['Primary Disease'])),px.colors.qualitative.Dark24))
subtype_colormap = dict(zip(list(set(node_lookup['Subtype Disease'])),list1))

In [56]:
node_lookup['Primary Color']=[primary_colormap[d] for d in node_lookup['Primary Disease']]
node_lookup['Subtype Color']=[subtype_colormap[d] for d in node_lookup['Subtype Disease']]

In [105]:
def calculate_positions(thresh_stack):

  # Create a networkx graph from the list of pairs
  G=nx.from_pandas_edgelist(thresh_stack, '#SOURCE', '#TARGET', ['#WEIGHT'])

  # Generate position data for each node:
  #pos=layout(G)
  # if weighted:
  pos=nx.spring_layout(G, weight='#WEIGHT')
      
  # Save x, y locations of each edge
  edge_x = []
  edge_y = []

  # Calculate x,y positions of an edge's 'start' (x0,y0) and 'end' (x1,y1) points
  for edge in G.edges():
      x0, y0 = pos[edge[0]]
      x1, y1 = pos[edge[1]]
      edge_x.append(x0)
      edge_x.append(x1)
      edge_y.append(y0)
      edge_y.append(y1)

  # Bundle it all up in a dict:
  edges = dict(x=edge_x,y=edge_y)

  # Save x, y locations of each node
  node_x = []
  node_y = []

  # Save node stats for annotation
  node_name = []
  node_adjacencies = []
  node_centralities = []

  # Calculate x,y positions of nodes
  for node in G.nodes():
      node_name.append(node)# Save node names
      x, y = pos[node]
      node_x.append(x)
      node_y.append(y)
    

  for node, adjacencies in enumerate(G.adjacency()):
      node_adjacencies.append(len(adjacencies[1]))

  for n in G.nodes():
      node_centralities.append(nx.degree_centrality(G)[n])

  # Bundle it all up in a dict:
  nodes = dict(x=node_x,y=node_y,name=node_name,adjacencies=node_adjacencies,centralities=node_centralities)

  return edges,nodes

In [106]:
[edges,nodes] = calculate_positions(network)

In [107]:
nodes['primary_color']=[list(node_lookup[node_lookup['Node_ID']==x]['Primary Color'])[0] for x in nodes['name']]

In [108]:
nodes['subtype_color']=[list(node_lookup[node_lookup['Node_ID']==x]['Subtype Color'])[0] for x in nodes['name']]

In [123]:
def draw_graph(edges,nodes,level,coloring,**kwargs):

  # Draw edges
  edge_trace = go.Scatter(
      x=edges['x'], y=edges['y'],
      line=dict(width=0.5, color='#888'),
      mode='lines+markers',
      hoverinfo='text')

  # Draw nodes
  node_trace = go.Scatter(
      x=nodes['x'],
      y=nodes['y'],
      # Optional: Add labels to points *without* hovering (can get a little messy)
      #mode='markers+text',
      # ...or, just add markers (no text)
      mode='markers',
      text=[str(list(node_lookup[node_lookup['Node_ID']==x]['Primary Disease'])[0]+": "+list(node_lookup[node_lookup['Node_ID']==x]["Subtype Disease"])[0]) for x in nodes['name']],
      hoverinfo='text')
  filename=level.lower().replace(" ","_")

  # Color the node by its number of connections
  node_trace.marker.color = nodes[coloring]
  node_trace.marker.size = 10
  node_trace.marker.line.color = nodes['primary_color']
  node_trace.marker.line.width = 3
  
  # Draw figure
  fig = go.Figure(data=[edge_trace,node_trace],
            layout=go.Layout(
              title='Network Graph of Cell Lines by '+level,
              height=700,
              titlefont_size=16,
              showlegend=False,
              hovermode='closest',
              margin=dict(b=20,l=5,r=5,t=120),
              xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
              yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
              template='plotly_white')
              )
  
  fig.update_traces(textposition='top center')
  # Show figure
  fig.show()

In [124]:
draw_graph(edges,nodes,'Primary Disease','primary_color')

In [125]:
draw_graph(edges,nodes,'Subtype Disease','subtype_color')